In [2]:
from datetime import date
from dateutil.relativedelta import relativedelta
from nsepy import*
from nsetools import *
import numpy
import pandas


#----- Global Variable  -----

StockName = ""


#----- Get 52 Week High Low -----

def Get52W_HL(P2y_To_P1y_EquityData):
    i = 0

    H_52W = P2y_To_P1y_EquityData['High'][i]
    L_52W = P2y_To_P1y_EquityData['Low'][i]

    while(i < len(P2y_To_P1y_EquityData['Close'])):
        if(H_52W < P2y_To_P1y_EquityData['High'][i]):
            H_52W = P2y_To_P1y_EquityData['High'][i]            
        
        if(L_52W > P2y_To_P1y_EquityData['Low'][i]):
            L_52W = P2y_To_P1y_EquityData['Low'][i]
        
        i = i + 1

    return {"52WH":H_52W,"52WL":L_52W}


#----- Add 52 Week High Low -----

def Add_52W_HL(EquityData,HL_52W):
    i = 0

    H_52W = HL_52W['52WH']
    L_52W = HL_52W['52WL']

    H_List = []
    L_List = []

    while(i < len(EquityData['Close'])):
        if(EquityData['High'][i] > H_52W):
            H_52W = EquityData['High'][i]
            H_List.append(EquityData['High'][i])     
        
        else:
            H_List.append(H_52W)

        if(EquityData['Low'][i] < L_52W):
            L_52W = EquityData['Low'][i]
            L_List.append(EquityData['Low'][i])
            
        else:
            L_List.append(L_52W)

        i = i + 1

    EquityData['52WH'] = H_List
    EquityData['52WL'] = L_List

    return EquityData


#----- Get  Equity Data -----

def GetEquityData_52W_HL():
    StartDate = date.today() - relativedelta(years=2)
    EndDate = date.today() - relativedelta(years=1)
    
    P2y_To_P1y_EquityData = get_history(symbol=StockName,start=StartDate,end=EndDate)

    HL_52W = Get52W_HL(P2y_To_P1y_EquityData)
    
    StartDate = date.today() - relativedelta(years=1)
    
    P1y_EquityData = get_history(symbol=StockName,start=StartDate,end=date.today())

    EquityData_52WHL = Add_52W_HL(P1y_EquityData,HL_52W)

    return EquityData_52WHL


#----- Percentage Change in Data -----

def PerChgInData(df):
    PCD = [0,]
    j=1

    while(j<len(df)):
        Val = ((df[j]-df[j-1]) / df[j-1])*100

        Change = round(Val,2)

        PCD.append(Change)
        
        j=j+1
    
    return PCD


#----- Get Delivery Value -----

def GetDelVal(df):
    i = 0

    DelVal = []

    while(i < len(df['VWAP'])):
        Val = (df['Deliverable Volume'][i] * df['VWAP'][i]) / 10000000

        DelVal.append(Val)

        i = i + 1

    return DelVal


#----- 5 Day Average Delivery -----

def  AvgDel_5(df):
    i = 4
    _AvgDel = [1,1,1,1,1]
    
    while(i<len(df)-1):
        j = i

        sum = 0

        while(j>=i-4):
            sum = sum + df[j]

            j=j-1
        
        avg = sum / 5
        _AvgDel.append(avg)

        i=i+1

    return _AvgDel


#----- List Devided by List -----

def LDL(df1,df2):
    i = 0
    _ldl =[]

    while(i<len(df1)):
        _ldl.append(round((df1[i] / df2[i])*100,2))

        i= i+ 1

    return _ldl

    

#----- Percentage Change in 52WH -----

def pcHigh(Close,H):
    i = 0
    pcH = []

    while(i < len(H)):
        Val = (H[i] - Close[i]) / H[i]

        Val = round(Val * 100 , 2)

        pcH.append(Val)

        i = i + 1

    return pcH


#----- Percentage Change in 52WH -----

def pcLow(Close,L):
    i = 0
    pcL = []

    while(i < len(L)):
        Val = (Close[i] - L[i]) / L[i]

        Val = round(Val * 100 , 2)

        pcL.append(Val)

        i = i + 1

    return pcL


#----- Operations -----

def Operations(df):
    df['Blank'] = ""

    df['Del Val'] = GetDelVal(df)

    df['5DAD'] = AvgDel_5(df['Del Val'])

    df['Blank2'] = ""
    
    df['Date2'] = df.index

    df['PC Price'] = PerChgInData(df['Close'])

    df['PC Del'] = LDL(df['Del Val'],df['5DAD'])

    df['Blank3'] = ""

    df['52WHigh'] = pcHigh(df['Close'],df['52WH'])

    df['52WLow'] = pcLow(df['Close'],df['52WL'])
    
    df['Blank4'] = ""

    df['VWAP2'] = df['VWAP']
    
    df['O'] = df['Open']

    df['H'] = df['High']

    df['L'] = df['Low']
    
    df['C'] = df['Close']

    return df


#----- Formatting Data Frame -----

def FormatDf(df):
    SD_Price = numpy.std(df['PC Price'])
    SD_Del = sum(df['PC Del'][4:]) / len(df['PC Del'][4:])
    
    
    df['PC Price'][0] = SD_Price
    df['PC Del'][0] = SD_Del
    
    
    FormatedDF = df.style\
        .applymap(lambda x: 'color: %s' % 'green; font-weight: bold' if x > SD_Price else 'background-colour: %s' % 'white', subset = ['PC Price'])\
        .applymap(lambda x: 'background-color: %s' % 'yellow' if x > SD_Del else 'background-colour: %s' % 'white', subset = ['PC Del'])\
        .applymap(lambda x: 'color: %s' % 'red;font-weight: bold' if x < -SD_Price else 'background-colour: %s' % 'white', subset = ['PC Price'])\
       
    return FormatedDF



#----- Main -----

def main():
    global StockName

    StockName = input("\nEnter the Symbol :")

    EquityData = GetEquityData_52W_HL()

    FinalDF = Operations(EquityData)

    FilePath = f'E:\kashy_mxe0hzp\zerodha\StocksExcel\\{StockName}{date.today()}.xlsx'
    FinalDF.to_excel(FilePath, engine='openpyxl', index=True)

    #print(EquityData.columns)

if __name__=="__main__":
    main()
    
FilePath = f'E:\kashy_mxe0hzp\zerodha\StocksExcel\\{StockName}{date.today()}.xlsx'
df = pandas. read_excel (FilePath, sheet_name='Sheet1' , engine='openpyxl')

FormatedDF = FormatDf(df)

FormatedDF
FormatedDF.to_excel(FilePath, engine='openpyxl', index=False)


Enter the Symbol :welspunind


<ipython-input-2-facb18a44af2>:242: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PC Price'][0] = SD_Price
<ipython-input-2-facb18a44af2>:243: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PC Del'][0] = SD_Del
